In [1]:
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.metrics import f1_score


In [2]:
# Load the training and test datasets
train_df = pd.read_parquet(r'C:\Users\aarek\etsydata\train')
test_df = pd.read_parquet(r'C:\Users\aarek\etsydata\test')


In [3]:
# Assuming train_df is already loaded and it's the correct DataFrame

# TF-IDF Vectorization for 'title' and 'description'
tfidf_title = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
title_features = tfidf_title.fit_transform(train_df['title'])

tfidf_description = TfidfVectorizer(max_features=300, ngram_range=(1, 2))
description_features = tfidf_description.fit_transform(train_df['description'])

# MultiLabelBinarizer for 'tags'
mlb = MultiLabelBinarizer()
tags_features = mlb.fit_transform(train_df['tags'].apply(lambda x: x.split(',')))

# LabelEncoder for 'type'
label_encoder = LabelEncoder()
type_features = label_encoder.fit_transform(train_df['type'])
type_features = type_features.reshape(-1, 1)  # Reshape for compatibility with hstack

# Combining all features into one feature matrix and converting to dense format
X_features = hstack([title_features, description_features, tags_features, type_features]).toarray()

# Labels
y = train_df['top_category_id']


MemoryError: Unable to allocate 823. GiB for an array with shape (229624, 962675) and data type int32

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)


In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)


In [ ]:
predictions = lda.predict(X_test)
f1 = f1_score(y_test, predictions, average='weighted')
print("F1 Score:", f1)
